In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import pandas as pd
import os
from pinecone import Pinecone, ServerlessSpec
import pinecone
from sentence_transformers import SentenceTransformer

In [ ]:
file = pd.read_csv("../resources/course_descriptions.csv", encoding="ANSI")

In [ ]:
file

In [ ]:
def create_course_description(row):
    return f"""The course name is {row['course_name']}, the slug is {row["course_slug"]}, the technology is {row["course_technology"]} and the course topic is {row['course_topic']}"""

In [ ]:
file['course_description'] = file.apply(create_course_description, axis=1)

In [ ]:
print(file['course_description'][0])

In [ ]:
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

In [ ]:
index_name = "my-index"
dimension=384
metric="cosine"

In [ ]:
#pc.create_index(name=index_name, dimension=dimension, metric=metric, spec=ServerlessSpec(cloud='aws', region='us-east-1'))

In [ ]:
index = pc.Index(index_name)

## Embedding the data

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def create_embeddings(row):
    combined_text = " ".join([str(row[field]) for field in ["course_description", "course_description_short"]])
    embedding = model.encode(combined_text, show_progress_bar=False)
    return embedding

In [ ]:
file['embedding'] = file.apply(create_embeddings, axis=1)

In [ ]:
vectors_to_upsert = [(str(row['course_name']), row['embedding'].tolist()) for _, row in file.iterrows()]
index.upsert(vectors=vectors_to_upsert)
print("Data upserted to Pinecone index")

## Semantic Search

In [ ]:
query = "clustering"
query_embedding = model.encode(query, show_progress_bar=False).tolist()

In [ ]:
query_results = index.query(vector=[query_embedding], top_k=12, include_values=True)

In [ ]:
query_results

In [ ]:
for match  in query_results["matches"]:
    if match['score'] >= 0.3:
        print(f"Matched item ID: {match['id']}, score: {match['score']}")

In [ ]:
file = pd.read_csv("../resources/course_section_descriptions.csv", encoding="ANSI")

In [ ]:
file['unique_id'] = file['course_id'].astype(str)+ '-' + file['section_id'].astype(str)

In [ ]:
file['metadata'] = file.apply(lambda row: {
    "course_name" : row['course_name'],
    "section_name" : row['section_name'],
    "section_description" : row['section_description'],
}, axis=1)

In [ ]:
file['metadata']

In [ ]:
def create_embeddings(row):
    combined_text = f'''{row["course_name"]} {row["course_technology"]}
                        {row["course_description"]} {row["section_name"]}{row["section_description"]}'''
    return model.encode(combined_text, show_progress_bar = False)

In [ ]:
file["embedding"] = file.apply(create_embeddings, axis = 1)

In [ ]:
vectors_to_upsert = [(row["unique_id"], row["embedding"].tolist(), row["metadata"]) for index, row in file.iterrows()  ]

In [ ]:
index.upsert(vectors=vectors_to_upsert)
print("Data is successfully upserted")

In [ ]:
query_results = index.query(
    vector = [query_embedding],
    top_k = 12,
    include_metadata=True
)

In [ ]:
score_threshold = 0.5

In [ ]:
# Assuming query_results are fetched and include metadata
for match in query_results['matches']:
    if match['score'] >= score_threshold:
        course_details = match.get('metadata', {})
        course_name = course_details.get('course_name', 'N/A')
        section_name = course_details.get('section_name', 'N/A')
        section_description = course_details.get('section_description', 'No description available')
        
        print(f"Matched item ID: {match['id']}, Score: {match['score']}")
        print(f"Course: {course_name} \nSection: {section_name} \nDescription: {section_description}\n")